In [13]:
import numpy as np
import six
import SimpleITK as sitk
from scipy.ndimage import zoom
import os,sys
import pandas as pd

from radiomics import featureextractor
import glob

In [2]:

imgDir = 'Data'
dirlist = os.listdir(imgDir)
print(dirlist)
print(len(dirlist))

['LIDC-IDRI-01', 'LIDC-IDRI-02', 'LIDC-IDRI-03', 'LIDC-IDRI-04', 'LIDC-IDRI-05', 'LIDC-IDRI-06', 'LIDC-IDRI-07', 'LIDC-IDRI-08', 'LIDC-IDRI-09', 'LIDC-IDRI-10', 'LIDC-IDRI-11', 'LIDC-IDRI-12', 'LIDC-IDRI-13', 'LIDC-IDRI-14', 'LIDC-IDRI-15', 'LIDC-IDRI-16', 'LIDC-IDRI-17', 'LIDC-IDRI-18', 'LIDC-IDRI-19', 'LIDC-IDRI-20', 'LIDC-IDRI-21', 'LIDC-IDRI-22', 'LIDC-IDRI-23', 'LIDC-IDRI-24', 'LIDC-IDRI-25', 'LIDC-IDRI-26', 'LIDC-IDRI-27', 'LIDC-IDRI-29', 'LIDC-IDRI-30', 'LIDC-IDRI-31', 'LIDC-IDRI-33', 'LIDC-IDRI-34', 'LIDC-IDRI-36', 'LIDC-IDRI-37', 'LIDC-IDRI-39', 'LIDC-IDRI-40', 'LIDC-IDRI-41', 'LIDC-IDRI-42', 'LIDC-IDRI-43', 'LIDC-IDRI-44', 'LIDC-IDRI-45', 'LIDC-IDRI-46', 'LIDC-IDRI-47', 'LIDC-IDRI-48', 'LIDC-IDRI-49', 'LIDC-IDRI-50']
46


In [7]:
# Feature Extraction
classification = {}
featureDict = {}
for ind in range(len(dirlist)):
    path = os.path.join(imgDir,dirlist[ind])
    imgpath = glob.glob(f'{path}/*.nrrd')
    img = sitk.ReadImage(imgpath[0])

    # pipeline to import data__________________
    # load image
    # log Segmentations for image
    path = os.path.join(path,'Segments')
    pathList = os.listdir(path)

    # #radiomics parameter
    params = 'Settings\Params.yaml'
    extractor = featureextractor.RadiomicsFeatureExtractor(params)

    for segID in range(len(pathList)):  #iterate over every segment
        segPath = os.path.join(imgDir,dirlist[ind],'Segments',pathList[segID])
        mask = sitk.ReadImage(segPath)

        # print(img.GetSize())
        # print(mask.GetSize())
        # print(img.GetOrigin())
        # print(mask.GetOrigin())
        # calculating features
        result = extractor.execute(img,mask)

        key = list(result.keys())
        key = key[1:]


        x = segPath.rfind('\\') #getting malignency

        feature = []
        for jind in range(len(key)):
            feature.append(result[key[jind]])

        if(segPath[x+1] in {'1','2'}):
            classification[dirlist[ind]+'_'+str(segID+1)] = 0
        else:
            classification[dirlist[ind]+'_'+str(segID+1)] = 1
        featureDict[dirlist[ind]+'_'+str(segID+1)] = feature
        dictkey = key
    print(dirlist[ind])


LIDC-IDRI-01


In [5]:
dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
dataframe.to_csv('Features_Radiomics.csv')

collections.OrderedDict